In [ ]:
import webbrowser
import urllib.parse
from dotenv import load_dotenv
from http.server import BaseHTTPRequestHandler, HTTPServer
import os
import requests
import base64
import json

In [ ]:
load_dotenv()
client_id = os.getenv("client_id")
redirect_uri = "http://127.0.0.1:8888/callback/"
scopes = "playlist-modify-private playlist-modify-public"


In [ ]:
auth_url = "https://accounts.spotify.com/authorize?" + urllib.parse.urlencode({
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": redirect_uri,
    "scope": scopes
})
auth_code = None
class CallbackHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        global auth_code
        query = urllib.parse.urlparse(self.path).query
        params = urllib.parse.parse_qs(query)
        if "code" in params:
            auth_code = params["code"][0]
            print("Authorization code:", auth_code)
            self.send_response(200)
            self.end_headers()
            self.wfile.write(b"You can close this window now.")
            # save the code somewhere or trigger token exchange
        else:
            self.send_response(400)
            self.end_headers()

server = HTTPServer(("127.0.0.1", 8888), CallbackHandler)
print("Waiting for Spotify redirect...")
webbrowser.open(auth_url)
print("Go to the URL above and log in.")
server.handle_request()
server.server_close()

In [ ]:
print(auth_code)

In [ ]:
client_secret = os.getenv("client_secret")

In [ ]:
def get_tokens(auth_code):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(
            f"{client_id}:{client_secret}".encode()
        ).decode()
    }
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri,
    }

    r = requests.post(url, headers=headers, data=data)
    r.raise_for_status()
    return r.json()

In [ ]:
r = get_tokens(auth_code)

In [ ]:
print(r)

In [ ]:
def refresh_access_token(refresh_token):
    url = "https://accounts.spotify.com/api/token"
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    headers = {"Authorization": f"Basic {auth_header}"}
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token
    }

    r = requests.post(url, headers=headers, data=data)
    r.raise_for_status()
    return r.json()

In [ ]:
refresh_token = os.getenv("refresh_token")
tokens = refresh_access_token(refresh_token)
print(tokens)

In [ ]:
access_token = refresh_access_token(refresh_token)['access_token']

In [ ]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
r = requests.get("https://api.spotify.com/v1/me", headers=headers)
r.raise_for_status()
user_id = r.json()["id"]

In [ ]:
r

In [ ]:
playlist_data = {
    "name": "My API Playlist",
    "description": "Created via Python CLI",
    "public": False
}

r = requests.post(f"https://api.spotify.com/v1/users/{user_id}/playlists",
                  headers=headers, data=json.dumps(playlist_data))
r.raise_for_status()
playlist = r.json()